In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sqlalchemy
from sqlalchemy import create_engine
from secrets import username, password

# UN Data #

In [2]:
#site to scrape
link = "https://en.wikipedia.org/wiki/List_of_countries_by_system_of_government"
tables = pd.read_html(link,header=0)

In [3]:
len(tables)

8

In [4]:
wiki_df5 = tables[5]

In [5]:
#Update casing for header row
wiki_df5.columns = [x.lower() for x in wiki_df5.columns ]

In [6]:
#Update country names
wiki_df5.loc[35,"Name"] = "China"

In [7]:
wiki_df5.loc[129,"Name"] = "Macedonia"

In [8]:
wiki_df5.loc[64,"Name"] = "Gambia"

In [9]:
wiki_df5.loc[91,"Name"] = "North Korea"

In [10]:
wiki_df5.loc[92,"Name"] = "South Korea"

In [11]:
wiki_df5.loc[38,"Name"] = "Democratic Republic of Congo"

In [12]:
wiki_df5.loc[39,"Name"] = "Republic of Congo"

In [13]:
wiki_df5.loc[154,"Name"] = "Republic of Serbia"

In [14]:
#add column name formatting
undercol = wiki_df5.rename(columns={'constitutional form': 'constitutional_form', 'head of state': 'head_of_state', 'basis of executive legitimacy': 'basis_of_executive_legitimacy' })
undercol

,name,constitutional_form,head_of_state,basis_of_executive_legitimacy,Name
0,Afghanistan,Republic,Executive,Presidency is independent of legislature,NaN
1,Albania,Republic,Ceremonial,Ministry is subject to parliamentary confidence,NaN
2,Algeria,Republic,Executive,Presidency independent of legislature; ministr...,NaN
3,Andorra,Constitutional monarchy,Ceremonial,Ministry is subject to parliamentary confidence,NaN
4,Angola,Republic,Executive,Presidency is independent of legislature,NaN
...,...,...,...,...,...
190,Venezuela,Republic,Executive,Presidency is independent of legislature,NaN
191,Vietnam,Republic,Executive,Power constitutionally linked to a single poli...,NaN
192,Yemen,Provisional,NaN,No constitutionally-defined basis to current r...,NaN
193,Zambia,Republic,Executive,Presidency is independent of legislature,NaN


In [15]:
#Drop countries not needed
undercol.drop(labels=[11, 151,24,189],axis=0)

,name,constitutional_form,head_of_state,basis_of_executive_legitimacy,Name
0,Afghanistan,Republic,Executive,Presidency is independent of legislature,NaN
1,Albania,Republic,Ceremonial,Ministry is subject to parliamentary confidence,NaN
2,Algeria,Republic,Executive,Presidency independent of legislature; ministr...,NaN
3,Andorra,Constitutional monarchy,Ceremonial,Ministry is subject to parliamentary confidence,NaN
4,Angola,Republic,Executive,Presidency is independent of legislature,NaN
...,...,...,...,...,...
190,Venezuela,Republic,Executive,Presidency is independent of legislature,NaN
191,Vietnam,Republic,Executive,Power constitutionally linked to a single poli...,NaN
192,Yemen,Provisional,NaN,No constitutionally-defined basis to current r...,NaN
193,Zambia,Republic,Executive,Presidency is independent of legislature,NaN


In [16]:
#Drop null column
clean_wiki =undercol.drop(labels=['Name'],axis=1)

In [17]:
#Replace na values
systems_govt = clean_wiki.fillna('No constitutionally-defined basis to current regime')

In [18]:
index = systems_govt['id'] = range(30, 30+len(systems_govt))

In [19]:
systems_govt.reset_index(drop=True)

,name,constitutional_form,head_of_state,basis_of_executive_legitimacy,id
0,Afghanistan,Republic,Executive,Presidency is independent of legislature,30
1,Albania,Republic,Ceremonial,Ministry is subject to parliamentary confidence,31
2,Algeria,Republic,Executive,Presidency independent of legislature; ministr...,32
3,Andorra,Constitutional monarchy,Ceremonial,Ministry is subject to parliamentary confidence,33
4,Angola,Republic,Executive,Presidency is independent of legislature,34
...,...,...,...,...,...
190,Venezuela,Republic,Executive,Presidency is independent of legislature,220
191,Vietnam,Republic,Executive,Power constitutionally linked to a single poli...,221
192,Yemen,Provisional,No constitutionally-defined basis to current r...,No constitutionally-defined basis to current r...,222
193,Zambia,Republic,Executive,Presidency is independent of legislature,223


In [20]:
systems_govt

,name,constitutional_form,head_of_state,basis_of_executive_legitimacy,id
0,Afghanistan,Republic,Executive,Presidency is independent of legislature,30
1,Albania,Republic,Ceremonial,Ministry is subject to parliamentary confidence,31
2,Algeria,Republic,Executive,Presidency independent of legislature; ministr...,32
3,Andorra,Constitutional monarchy,Ceremonial,Ministry is subject to parliamentary confidence,33
4,Angola,Republic,Executive,Presidency is independent of legislature,34
...,...,...,...,...,...
190,Venezuela,Republic,Executive,Presidency is independent of legislature,220
191,Vietnam,Republic,Executive,Power constitutionally linked to a single poli...,221
192,Yemen,Provisional,No constitutionally-defined basis to current r...,No constitutionally-defined basis to current r...,222
193,Zambia,Republic,Executive,Presidency is independent of legislature,223


# happiness data #

In [21]:
# load csv

happiness_data_to_load = "./raw_data/WHR20_DataForFigure2.1.csv"
happiness_df = pd.read_csv(happiness_data_to_load)
happiness_df

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,3.4759,0.115183,3.701658,3.250141,6.625160,0.319460,45.200001,0.640881,0.082410,0.891807,1.972317,0.041072,0.000000,0.000000,0.292814,0.253513,0.028265,2.860198
149,Rwanda,Sub-Saharan Africa,3.3123,0.052425,3.415053,3.209547,7.600104,0.540835,61.098846,0.900589,0.055484,0.183541,1.972317,0.343243,0.522876,0.572383,0.604088,0.235705,0.485542,0.548445
150,Zimbabwe,Sub-Saharan Africa,3.2992,0.058674,3.414202,3.184198,7.865712,0.763093,55.617260,0.711458,-0.072064,0.810237,1.972317,0.425564,1.047835,0.375038,0.377405,0.151349,0.080929,0.841031
151,South Sudan,Sub-Saharan Africa,2.8166,0.107610,3.027516,2.605684,7.425360,0.553707,51.000000,0.451314,0.016519,0.763417,1.972317,0.289083,0.553279,0.208809,0.065609,0.209935,0.111157,1.378751


In [22]:
# grab columns we need
happiness_df_drop = happiness_df.drop(columns = ['Standard error of ladder score', 'upperwhisker', 'lowerwhisker', 'Logged GDP per capita', 'Explained by: Log GDP per capita', 'Explained by: Social support', 'Explained by: Healthy life expectancy', 'Explained by: Freedom to make life choices', 'Explained by: Generosity', 'Explained by: Perceptions of corruption', 'Dystopia + residual'])
happiness_df_drop

,Country name,Regional indicator,Ladder score,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia
0,Finland,Western Europe,7.8087,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317
1,Denmark,Western Europe,7.6456,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317
2,Switzerland,Western Europe,7.5599,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317
3,Iceland,Western Europe,7.5045,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317
4,Norway,Western Europe,7.4880,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317
...,...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,3.4759,0.319460,45.200001,0.640881,0.082410,0.891807,1.972317
149,Rwanda,Sub-Saharan Africa,3.3123,0.540835,61.098846,0.900589,0.055484,0.183541,1.972317
150,Zimbabwe,Sub-Saharan Africa,3.2992,0.763093,55.617260,0.711458,-0.072064,0.810237,1.972317
151,South Sudan,Sub-Saharan Africa,2.8166,0.553707,51.000000,0.451314,0.016519,0.763417,1.972317


In [23]:
# count values in each column

happiness_df_drop.count()

Country name                    153
Regional indicator              153
Ladder score                    153
Social support                  153
Healthy life expectancy         153
Freedom to make life choices    153
Generosity                      153
Perceptions of corruption       153
Ladder score in Dystopia        153
dtype: int64

In [24]:
# rename columns
happiness_df_renamed = happiness_df_drop.rename(columns={"Country name": "country", "Regional indicator": "world_region", "Ladder score": "ladder_score", "Social support": "social_support", "Healthy life expectancy": "healthy_life_expectancy", "Freedom to make life choices": "freedom_of_choice", "Generosity": "generosity", "Perceptions of corruption": "perceptions_of_corruption", "Ladder score in Dystopia": "ladder_score_in_dystopia"})
happiness_df_renamed

,country,world_region,ladder_score,social_support,healthy_life_expectancy,freedom_of_choice,generosity,perceptions_of_corruption,ladder_score_in_dystopia
0,Finland,Western Europe,7.8087,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317
1,Denmark,Western Europe,7.6456,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317
2,Switzerland,Western Europe,7.5599,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317
3,Iceland,Western Europe,7.5045,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317
4,Norway,Western Europe,7.4880,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317
...,...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,3.4759,0.319460,45.200001,0.640881,0.082410,0.891807,1.972317
149,Rwanda,Sub-Saharan Africa,3.3123,0.540835,61.098846,0.900589,0.055484,0.183541,1.972317
150,Zimbabwe,Sub-Saharan Africa,3.2992,0.763093,55.617260,0.711458,-0.072064,0.810237,1.972317
151,South Sudan,Sub-Saharan Africa,2.8166,0.553707,51.000000,0.451314,0.016519,0.763417,1.972317


In [25]:
# rename countries to match

happiness_df_renamed.loc[132,"country"] = "Democratic Republic of Congo"
happiness_df_renamed.loc[89,"country"] = "Republic of Congo"
happiness_df_renamed.loc[65,"country"] = "Republic of Serbia"

In [26]:
# drop unrecognized states

happiness_df_clean = happiness_df_renamed.drop([24, 77, 131])
happiness_df_clean

,country,world_region,ladder_score,social_support,healthy_life_expectancy,freedom_of_choice,generosity,perceptions_of_corruption,ladder_score_in_dystopia
0,Finland,Western Europe,7.8087,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317
1,Denmark,Western Europe,7.6456,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317
2,Switzerland,Western Europe,7.5599,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317
3,Iceland,Western Europe,7.5045,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317
4,Norway,Western Europe,7.4880,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317
...,...,...,...,...,...,...,...,...,...
148,Central African Republic,Sub-Saharan Africa,3.4759,0.319460,45.200001,0.640881,0.082410,0.891807,1.972317
149,Rwanda,Sub-Saharan Africa,3.3123,0.540835,61.098846,0.900589,0.055484,0.183541,1.972317
150,Zimbabwe,Sub-Saharan Africa,3.2992,0.763093,55.617260,0.711458,-0.072064,0.810237,1.972317
151,South Sudan,Sub-Saharan Africa,2.8166,0.553707,51.000000,0.451314,0.016519,0.763417,1.972317


In [27]:
happiness_df_clean.insert(0, 'country_id', range(30, 30 + len(happiness_df_clean)))
happiness_df_clean

,country_id,country,world_region,ladder_score,social_support,healthy_life_expectancy,freedom_of_choice,generosity,perceptions_of_corruption,ladder_score_in_dystopia
0,30,Finland,Western Europe,7.8087,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317
1,31,Denmark,Western Europe,7.6456,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317
2,32,Switzerland,Western Europe,7.5599,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317
3,33,Iceland,Western Europe,7.5045,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317
4,34,Norway,Western Europe,7.4880,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317
...,...,...,...,...,...,...,...,...,...,...
148,175,Central African Republic,Sub-Saharan Africa,3.4759,0.319460,45.200001,0.640881,0.082410,0.891807,1.972317
149,176,Rwanda,Sub-Saharan Africa,3.3123,0.540835,61.098846,0.900589,0.055484,0.183541,1.972317
150,177,Zimbabwe,Sub-Saharan Africa,3.2992,0.763093,55.617260,0.711458,-0.072064,0.810237,1.972317
151,178,South Sudan,Sub-Saharan Africa,2.8166,0.553707,51.000000,0.451314,0.016519,0.763417,1.972317


# covid data #

In [28]:
# import csv as df

covid_data_to_load = "./raw_data/owid-covid-data.csv"
covid_df_raw = pd.read_csv(covid_data_to_load)
covid_df_raw

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66962,ZWE,Africa,Zimbabwe,2021-02-01,33548.0,160.0,271.714,1234.0,17.0,22.714,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
66963,ZWE,Africa,Zimbabwe,2021-02-02,33814.0,266.0,258.571,1254.0,20.0,21.571,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
66964,ZWE,Africa,Zimbabwe,2021-02-03,33964.0,150.0,237.143,1269.0,15.0,21.000,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
66965,ZWE,Africa,Zimbabwe,2021-02-04,34171.0,207.0,217.857,1288.0,19.0,18.286,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535


In [29]:
# grab columns we want
covid_df = covid_df_raw[['location', 'date', 'new_cases', 'new_deaths', 'icu_patients']]
covid_df

,location,date,new_cases,new_deaths,icu_patients
0,Afghanistan,2020-02-24,1.0,NaN,NaN
1,Afghanistan,2020-02-25,0.0,NaN,NaN
2,Afghanistan,2020-02-26,0.0,NaN,NaN
3,Afghanistan,2020-02-27,0.0,NaN,NaN
4,Afghanistan,2020-02-28,0.0,NaN,NaN
...,...,...,...,...,...
66962,Zimbabwe,2021-02-01,160.0,17.0,NaN
66963,Zimbabwe,2021-02-02,266.0,20.0,NaN
66964,Zimbabwe,2021-02-03,150.0,15.0,NaN
66965,Zimbabwe,2021-02-04,207.0,19.0,NaN


In [30]:
# rename columns
covid_df = covid_df.rename(columns={"location": "country"})
covid_df

,country,date,new_cases,new_deaths,icu_patients
0,Afghanistan,2020-02-24,1.0,NaN,NaN
1,Afghanistan,2020-02-25,0.0,NaN,NaN
2,Afghanistan,2020-02-26,0.0,NaN,NaN
3,Afghanistan,2020-02-27,0.0,NaN,NaN
4,Afghanistan,2020-02-28,0.0,NaN,NaN
...,...,...,...,...,...
66962,Zimbabwe,2021-02-01,160.0,17.0,NaN
66963,Zimbabwe,2021-02-02,266.0,20.0,NaN
66964,Zimbabwe,2021-02-03,150.0,15.0,NaN
66965,Zimbabwe,2021-02-04,207.0,19.0,NaN


In [31]:
# look at types

covid_df.dtypes

country          object
date             object
new_cases       float64
new_deaths      float64
icu_patients    float64
dtype: object

In [32]:
# look at duplicates

covid_df.duplicated(keep=False).count()

66967

In [33]:
# count values in each column

covid_df.count()

country         66967
date            66967
new_cases       66354
new_deaths      57688
icu_patients     7167
dtype: int64

In [34]:
# replace blanks with NaN values

nan_covid_df = covid_df.replace(r'', np.nan)

In [35]:
# replace NaN values with 0s

zero_covid_df = nan_covid_df.replace(np.nan, 0)
zero_covid_df

,country,date,new_cases,new_deaths,icu_patients
0,Afghanistan,2020-02-24,1.0,0.0,0.0
1,Afghanistan,2020-02-25,0.0,0.0,0.0
2,Afghanistan,2020-02-26,0.0,0.0,0.0
3,Afghanistan,2020-02-27,0.0,0.0,0.0
4,Afghanistan,2020-02-28,0.0,0.0,0.0
...,...,...,...,...,...
66962,Zimbabwe,2021-02-01,160.0,17.0,0.0
66963,Zimbabwe,2021-02-02,266.0,20.0,0.0
66964,Zimbabwe,2021-02-03,150.0,15.0,0.0
66965,Zimbabwe,2021-02-04,207.0,19.0,0.0


In [36]:
# look at value types

zero_covid_df.dtypes

country          object
date             object
new_cases       float64
new_deaths      float64
icu_patients    float64
dtype: object

In [37]:
# remove 2021 dates

clean_covid_df = zero_covid_df[(zero_covid_df['date'] < '2021-01-01')]

In [38]:
clean_covid_df.count()

country         59804
date            59804
new_cases       59804
new_deaths      59804
icu_patients    59804
dtype: int64

In [39]:
# convert date to datetime value
clean_covid_df['date'] = pd.to_datetime(clean_covid_df['date'])
clean_covid_df['date'] = clean_covid_df['date'].dt.strftime('%m-%d-%Y')
clean_covid_df['date'] = pd.to_datetime(clean_covid_df['date'])

<ipython-input-39-a8fd057f9218>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_covid_df['date'] = pd.to_datetime(clean_covid_df['date'])
<ipython-input-39-a8fd057f9218>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_covid_df['date'] = clean_covid_df['date'].dt.strftime('%m-%d-%Y')
<ipython-input-39-a8fd057f9218>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [40]:
clean_covid_df.count()

country         59804
date            59804
new_cases       59804
new_deaths      59804
icu_patients    59804
dtype: int64

In [41]:
# rename countries

clean_covid_df.loc[13680:14007,"country"] = "Republic of Congo"
clean_covid_df.loc[44324:44669,"country"] = "Macedonia"

C:\Users\wok2k\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [42]:
# drop countries not in UN

clean_covid_df = clean_covid_df[clean_covid_df.country != "Taiwan"]
clean_covid_df = clean_covid_df[clean_covid_df.country != "Bahamas"]
clean_covid_df = clean_covid_df[clean_covid_df.country != "Hong Kong"]
clean_covid_df = clean_covid_df[clean_covid_df.country != "International"]

In [43]:
clean_covid_df.insert(0, 'covid_cases_id', range(30, 30 + len(clean_covid_df)))

In [44]:
clean_covid_df

,covid_cases_id,country,date,new_cases,new_deaths,icu_patients
0,30,Afghanistan,2020-02-24,1.0,0.0,0.0
1,31,Afghanistan,2020-02-25,0.0,0.0,0.0
2,32,Afghanistan,2020-02-26,0.0,0.0,0.0
3,33,Afghanistan,2020-02-27,0.0,0.0,0.0
4,34,Afghanistan,2020-02-28,0.0,0.0,0.0
...,...,...,...,...,...,...
66926,58787,Zimbabwe,2020-12-27,114.0,8.0,0.0
66927,58788,Zimbabwe,2020-12-28,71.0,5.0,0.0
66928,58789,Zimbabwe,2020-12-29,177.0,5.0,0.0
66929,58790,Zimbabwe,2020-12-30,300.0,1.0,0.0


# govt response data #

In [45]:
# File to path
gov_responses_path = './raw_data/coronanet_release.csv'
# Read csv and store in to Pandas Dataframe
gov_df = pd.read_csv(gov_responses_path)

In [46]:
# Take a look at the dataframe
gov_df = pd.DataFrame(gov_df)
gov_df.head()

,record_id,policy_id,entry_type,correct_type,update_type,update_level,description,date_announced,date_start,date_end,...,travel_mechanism,compliance,enforcer,index_high_est,index_med_est,index_low_est,index_country_rank,link,date_updated,recorded_date
0,R_3nCudDknlhUIjpODy,2975738,new_entry,original,NaN,NaN,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,2020-03-06,NaN,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp...",53.029449,50.604971,48.280188,137.0,https://www.etilaatroz.com/94246/fears-rumors-...,2020-08-21,2020-08-21T18:32:29Z
1,R_3nCudDknlhUIjpOBt,2975738,new_entry,original,NaN,NaN,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,2020-03-06,NaN,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp...",53.029449,50.604971,48.280188,137.0,https://www.etilaatroz.com/94246/fears-rumors-...,2020-08-21,2020-08-21T18:32:29Z
2,R_3nCudDknlhUIjpOBg,2975738,new_entry,original,NaN,NaN,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,2020-03-06,NaN,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp...",53.029449,50.604971,48.280188,137.0,https://www.etilaatroz.com/94246/fears-rumors-...,2020-08-21,2020-08-21T18:32:29Z
3,R_3nCudDknlhUIjpOCg,2975738,new_entry,original,NaN,NaN,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,2020-03-06,NaN,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp...",53.029449,50.604971,48.280188,137.0,https://www.etilaatroz.com/94246/fears-rumors-...,2020-08-21,2020-08-21T18:32:29Z
4,R_3nCudDknlhUIjpOCw,2975738,new_entry,original,NaN,NaN,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,2020-03-06,NaN,...,NaN,Voluntary/Recommended but No Penalties,"Ministry/Department of Health,Other (Please sp...",53.029449,50.604971,48.280188,137.0,https://www.etilaatroz.com/94246/fears-rumors-...,2020-08-21,2020-08-21T18:32:29Z


In [47]:
# look at all the column headers
for col in gov_df.columns: 
    print(col)

record_id
policy_id
entry_type
correct_type
update_type
update_level
description
date_announced
date_start
date_end
country
ISO_A3
ISO_A2
init_country_level
domestic_policy
province
city
type
type_sub_cat
type_text
school_status
target_country
target_geog_level
target_region
target_province
target_city
target_other
target_who_what
target_direction
travel_mechanism
compliance
enforcer
index_high_est
index_med_est
index_low_est
index_country_rank
link
date_updated
recorded_date


In [48]:
# Drop columns that I can see
gov_df_drop = gov_df.drop(columns=["update_type","record_id","entry_type","correct_type","update_type","update_level", "travel_mechanism", "index_high_est","index_med_est","index_low_est","index_country_rank","date_updated","recorded_date"])

In [49]:
# Drop additional columns
gov_df_drop = gov_df_drop.drop(columns=["ISO_A3","ISO_A2","init_country_level","domestic_policy","target_geog_level","target_region","target_province","target_city","target_other","target_direction"])

In [50]:
# drop columns not to be used
gov_df_drop = gov_df_drop.drop(columns=['province',"type_text","school_status","target_country","date_end","date_announced","city", "compliance","target_who_what","type_sub_cat"])

In [51]:
# Drop Duplicate rows that had were tied together by different variables found in the school_status column that was dropped earlier - 
#schools_status had either primary or secondary listed
gov_df_drop=gov_df_drop.drop_duplicates()
gov_df_drop.head()

,policy_id,description,date_start,country,type,enforcer,link
0,2975738,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
17,889484,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
34,129322,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
51,6721435,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
53,6018486,"In parts of northern Samangan province, school...",2020-03-11,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://www.pajhwok.com/en/2020/03/11/samangan...


In [52]:
# take a look at dtypes
gov_df_drop.dtypes

policy_id       int64
description    object
date_start     object
country        object
type           object
enforcer       object
link           object
dtype: object

In [53]:
# Count duplicates
gov_df_drop.duplicated(keep=False).count()

28385

In [54]:
# Count how many rows in each column
gov_df_drop.count()

policy_id      28385
description    28385
date_start     28385
country        28385
type           28385
enforcer       28342
link           28291
dtype: int64

In [55]:
# Keeping urls with http links
gov_df_drop=gov_df_drop.dropna(subset=['link'])
gov_df_drop.tail()

,policy_id,description,date_start,country,type,enforcer,link
61032,3887260,Venues hosting low-risk sport competitions in ...,2020-05-17,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Police",http://www.veritaszim.net/sites/veritas_d/file...
61049,3619429,"As of May 22, Zimbabwe has implemented a reduc...",2020-05-22,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Ministry/Department of Hea...",https://www.herald.co.zw/nurses-flexi-hours-re...
61066,2531192,A presidential amnesty was announced by the Zi...,2020-03-02,Zimbabwe,Restrictions of Mass Gatherings,"National Government,Ministry/Department of Jus...",https://ewn.co.za/2020/03/04/zimbabwe-to-relea...
61067,2574281,"In Zimbabwe, the 40th independence anniversary...",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,National Government,http://apanews.net/en/pays/zimbabwe/news/covid...
61068,6689905,"In Zimbabwe, the Zimbabwe International Trade ...",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,National Government,https://iharare.com/independence-day-zitf-post...


In [56]:
# check count
gov_df_drop.count()

policy_id      28291
description    28291
date_start     28291
country        28291
type           28291
enforcer       28256
link           28291
dtype: int64

In [57]:
# Keeping urls with http links
gov_df_drop=gov_df_drop.dropna(subset=['link'])
gov_df_drop

,policy_id,description,date_start,country,type,enforcer,link
0,2975738,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
17,889484,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
34,129322,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
51,6721435,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
53,6018486,"In parts of northern Samangan province, school...",2020-03-11,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://www.pajhwok.com/en/2020/03/11/samangan...
...,...,...,...,...,...,...,...
61032,3887260,Venues hosting low-risk sport competitions in ...,2020-05-17,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Police",http://www.veritaszim.net/sites/veritas_d/file...
61049,3619429,"As of May 22, Zimbabwe has implemented a reduc...",2020-05-22,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Ministry/Department of Hea...",https://www.herald.co.zw/nurses-flexi-hours-re...
61066,2531192,A presidential amnesty was announced by the Zi...,2020-03-02,Zimbabwe,Restrictions of Mass Gatherings,"National Government,Ministry/Department of Jus...",https://ewn.co.za/2020/03/04/zimbabwe-to-relea...
61067,2574281,"In Zimbabwe, the 40th independence anniversary...",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,National Government,http://apanews.net/en/pays/zimbabwe/news/covid...


In [58]:
#check count
gov_df_drop.count()

policy_id      28291
description    28291
date_start     28291
country        28291
type           28291
enforcer       28256
link           28291
dtype: int64

In [59]:
# Drop rows that do not contain a URL
gov_df_drop=gov_df_drop[gov_df_drop['link'].str.contains('http')]
gov_df_drop.head()

,policy_id,description,date_start,country,type,enforcer,link
0,2975738,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
17,889484,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
34,129322,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
51,6721435,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
53,6018486,"In parts of northern Samangan province, school...",2020-03-11,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://www.pajhwok.com/en/2020/03/11/samangan...


In [60]:
# check count
gov_df_drop.count()

policy_id      28109
description    28109
date_start     28109
country        28109
type           28109
enforcer       28074
link           28109
dtype: int64

In [61]:
# Keeping rows that have a enforcer mentioned
gov_df_drop=gov_df_drop.dropna(subset=['enforcer'])
gov_df_drop.head()

,policy_id,description,date_start,country,type,enforcer,link
0,2975738,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
17,889484,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
34,129322,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
51,6721435,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
53,6018486,"In parts of northern Samangan province, school...",2020-03-11,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://www.pajhwok.com/en/2020/03/11/samangan...


In [62]:
# Check count
gov_df_drop.count()

policy_id      28074
description    28074
date_start     28074
country        28074
type           28074
enforcer       28074
link           28074
dtype: int64

In [63]:
# Drop rows that contain nothing in the description column
gov_df_drop=gov_df_drop[gov_df_drop['description'].str.contains(' ')]
gov_df_drop.head()

,policy_id,description,date_start,country,type,enforcer,link
0,2975738,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
17,889484,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
34,129322,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
51,6721435,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
53,6018486,"In parts of northern Samangan province, school...",2020-03-11,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://www.pajhwok.com/en/2020/03/11/samangan...


In [64]:
# keep the last row of duplicates
clean_gov_df=gov_df_drop.drop_duplicates(subset=['policy_id'], keep='last')
clean_gov_df

,policy_id,description,date_start,country,type,enforcer,link
0,2975738,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
17,889484,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
34,129322,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
51,6721435,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
57,3558044,"""On 14 March, the Government of Afghanistan an...",2020-03-14,Afghanistan,Closure and Regulation of Schools,National Government,https://reliefweb.int/report/afghanistan/afgha...
...,...,...,...,...,...,...,...
61032,3887260,Venues hosting low-risk sport competitions in ...,2020-05-17,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Police",http://www.veritaszim.net/sites/veritas_d/file...
61049,3619429,"As of May 22, Zimbabwe has implemented a reduc...",2020-05-22,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Ministry/Department of Hea...",https://www.herald.co.zw/nurses-flexi-hours-re...
61066,2531192,A presidential amnesty was announced by the Zi...,2020-03-02,Zimbabwe,Restrictions of Mass Gatherings,"National Government,Ministry/Department of Jus...",https://ewn.co.za/2020/03/04/zimbabwe-to-relea...
61067,2574281,"In Zimbabwe, the 40th independence anniversary...",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,National Government,http://apanews.net/en/pays/zimbabwe/news/covid...


In [65]:
# Drop Policy ID column and reset index for index numbers to go in sequential order
clean_gov_df= clean_gov_df.drop(columns=["policy_id"]).reset_index()
clean_gov_df= clean_gov_df.drop(columns=["index"])
clean_gov_df.head()

,description,date_start,country,type,enforcer,link
0,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
1,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
2,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
3,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
4,"""On 14 March, the Government of Afghanistan an...",2020-03-14,Afghanistan,Closure and Regulation of Schools,National Government,https://reliefweb.int/report/afghanistan/afgha...


In [66]:
# rename columns
clean_gov_df = clean_gov_df.rename(columns={'description': 'gov_resp_desc', 'date_start': 'gov_resp_date', 'type':'gov_resp_type','link':'gov_resp_link'})
clean_gov_df.head()

,gov_resp_desc,gov_resp_date,country,gov_resp_type,enforcer,gov_resp_link
0,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,"Ministry/Department of Health,Other (Please sp...",https://www.etilaatroz.com/94246/fears-rumors-...
1,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
2,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,"National Government,Other (Please specify in t...",https://www.worldbank.org/en/news/feature/2020...
3,"""All schools in Herat – including temporarily ...",2020-03-09,Afghanistan,Closure and Regulation of Schools,Provincial/State Government,https://reliefweb.int/sites/reliefweb.int/file...
4,"""On 14 March, the Government of Afghanistan an...",2020-03-14,Afghanistan,Closure and Regulation of Schools,National Government,https://reliefweb.int/report/afghanistan/afgha...


In [67]:
# check for date to fix year
clean_gov_df.loc[19440: 19442]

,gov_resp_desc,gov_resp_date,country,gov_resp_type,enforcer,gov_resp_link
19440,"Pennsylvania, USA, 14th July: Dept of Human Se...",2020-08-14,United States of America,"New Task Force, Bureau or Administrative Confi...",Other (Please specify in the text box),https://www.media.pa.gov/pages/DHS_details.asp...
19441,The Michigan Nursing Homes COVID-19 Preparedne...,2020-08-26,United States of America,"New Task Force, Bureau or Administrative Confi...","Ministry/Department of Health,Provincial/State...",https://content.govdelivery.com/attachments/MI...
19442,US state Michigan creates new task force for e...,2024-05-29,United States of America,"New Task Force, Bureau or Administrative Confi...",New (COVID-19 specific) Task Force,"https://www.michigan.gov/whitmer/0,9309,7-387-..."


In [68]:
clean_gov_df.at[19442,"gov_resp_date"]= "2020-05-29"

In [69]:
# change date object to datetime
clean_gov_df['gov_resp_date'] = pd.to_datetime(clean_gov_df['gov_resp_date'])
clean_gov_df['gov_resp_date'] = clean_gov_df['gov_resp_date'].dt.strftime('%m-%d-%Y')
clean_gov_df['gov_resp_date'] = pd.to_datetime(clean_gov_df['gov_resp_date'])

In [70]:
# check datetime
clean_gov_df.dtypes

gov_resp_desc            object
gov_resp_date    datetime64[ns]
country                  object
gov_resp_type            object
enforcer                 object
gov_resp_link            object
dtype: object

In [71]:
# check count
clean_gov_df.count()

gov_resp_desc    21285
gov_resp_date    21285
country          21285
gov_resp_type    21285
enforcer         21285
gov_resp_link    21285
dtype: int64

In [72]:
# change name of dataframe
gov_response = clean_gov_df
gov_response.tail()

,gov_resp_desc,gov_resp_date,country,gov_resp_type,enforcer,gov_resp_link
21280,Venues hosting low-risk sport competitions in ...,2020-05-17,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Police",http://www.veritaszim.net/sites/veritas_d/file...
21281,"As of May 22, Zimbabwe has implemented a reduc...",2020-05-22,Zimbabwe,Restriction and Regulation of Government Services,"National Government,Ministry/Department of Hea...",https://www.herald.co.zw/nurses-flexi-hours-re...
21282,A presidential amnesty was announced by the Zi...,2020-03-02,Zimbabwe,Restrictions of Mass Gatherings,"National Government,Ministry/Department of Jus...",https://ewn.co.za/2020/03/04/zimbabwe-to-relea...
21283,"In Zimbabwe, the 40th independence anniversary...",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,National Government,http://apanews.net/en/pays/zimbabwe/news/covid...
21284,"In Zimbabwe, the Zimbabwe International Trade ...",2020-03-17,Zimbabwe,Restrictions of Mass Gatherings,National Government,https://iharare.com/independence-day-zitf-post...


In [73]:
# look at enforcer column and check for odd unique values to remove 
gov_response.enforcer.unique()

array(['Ministry/Department of Health,Other (Please specify in the text box)',
       'National Government,Other (Please specify in the text box)',
       'Provincial/State Government', 'National Government',
       'National Government,Ministry/Department of Health,Other (Please specify in the text box)',
       'Provincial/State Government,Other (Please specify in the text box)',
       'Ministry/Department of Health,Provincial/State Government,New (COVID-19 specific) Task Force',
       'Ministry/Department of Health,Provincial/State Government,Other (Please specify in the text box)',
       'National Government,Ministry/Department of Health,Provincial/State Government,Police,Other (Please specify in the text box)',
       'Ministry/Department of Health',
       'National Government,Ministry/Department of Health',
       'National Government,Ministry/Department of Health,Provincial/State Government',
       'National Government,Provincial/State Government',
       'Provincial/State 

In [74]:
# take out string from enforcer column - (Please specify in the text box)
gov_response['enforcer'] = gov_response['enforcer'].map(lambda x: x.rstrip('(Please specify in the text box)'))
gov_response.enforcer.unique()

array(['Ministry/Department of Health,Other', 'National Government,Other',
       'Provincial/State Governm', 'National Governm',
       'National Government,Ministry/Department of Health,Other',
       'Provincial/State Government,Other',
       'Ministry/Department of Health,Provincial/State Government,New (COVID-19 specific) Task For',
       'Ministry/Department of Health,Provincial/State Government,Other',
       'National Government,Ministry/Department of Health,Provincial/State Government,Police,Other',
       'Ministry/Department of H',
       'National Government,Ministry/Department of H',
       'National Government,Ministry/Department of Health,Provincial/State Governm',
       'National Government,Provincial/State Governm',
       'Provincial/State Government,Municipal Governm',
       'Ministry/Department of Health,Provincial/State Governm',
       'Ministry/Department of Health,Provincial/State Government,Police,Other',
       'National Government,Ministry/Department of H

In [75]:
# Check for NaN 
gov_response.isnull().values.any()

False

In [76]:
# look at all the countries
gov_response.country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei',
       'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon',
       'Canada', 'Cape Verde', 'Central African Republic', 'Chad',
       'Chile', 'China', 'Colombia', 'Comoros', 'Costa Rica',
       'Côte d’Ivoire', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Ethiopia', 'European Union', 'Federated States of Micronesia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Grenada', 'Gua

In [77]:
# delete rows that have countries not in other datasets - keeping it consistent
gov_response = gov_response[gov_response.country != 'Jersey']
gov_response = gov_response[gov_response.country != 'Taiwan']
gov_response = gov_response[gov_response.country != 'Ivory Coast']
gov_response = gov_response[gov_response.country != 'Swaziland']
gov_response = gov_response[gov_response.country != 'Republic of Serbia']
gov_response = gov_response[gov_response.country != 'The Bahamas']
gov_response = gov_response[gov_response.country != 'Brunei']
gov_response = gov_response[gov_response.country != 'Vatican']
gov_response = gov_response[gov_response.country != 'Sao Tome and Principe']
gov_response = gov_response[gov_response.country != 'São Tomé & Príncipe']

In [78]:
# look at all the countries
gov_response.country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Bulgaria',
       'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada',
       'Cape Verde', 'Central African Republic', 'Chad', 'Chile', 'China',
       'Colombia', 'Comoros', 'Costa Rica', 'Côte d’Ivoire', 'Croatia',
       'Cuba', 'Cyprus', 'Czech Republic',
       'Democratic Republic of the Congo', 'Denmark', 'Djibouti',
       'Dominica', 'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Ethiopia', 'European Union', 'Federated States of Micronesia',
       'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', '

# SQL Alchemy #

In [79]:
# from sqlalchemy import create_engine

# engine = create_engine('postgresql://postgres:<insert password>@localhost:5432/Happiness_db')
# conn=engine.connect()

# gov_response.to_sql(name='gov_response',con=engine, if_exists='append',index=False)

In [80]:
#rds_connection_string = f"{username}:{password}@localhost:5432/customer_db"
#engine = create_engine(f'postgresql://{rds_connection_string}')


In [81]:
# gov_type = pd.read_sql("SELECT gov_resp_type FROM gov_response", conn )
# gov_type

In [82]:
# conn.close()